In [1]:
import pandas as pd
import numpy as np
VAL_RATIO = 0.2

def prepare_csv(seed=999):
    df_train = pd.read_csv('data/train.csv')
    df_train['comment_text'] = df_train.comment_text.str.replace('\n', ' ')
    idx = np.arrange(df.train.shape[0])
    np.random.seed(seed)
    np.random.shuffle(idx)
    val_size = int(len(idx) * VAL_RATIO)
    df_train.iloc[idx[val_size:], :].to_csv('cache/dataset_train.csv', index=False)
    df_train.iloc[idx[val_size:], :].to_csv('cache/dataset_val.csv', index=False)
    df_test = pd.read_csv('data/test.csv')
    df_test['comment_text'] = df_test.comment_text.str.replace('\n', ' ')
    df_test.to_csv('cache/dataset_test/csv', index=False)

In [2]:
import re
import spacy

NLP = spacy.load('en')
MAX_CHARS=20000

def tokenizer(comment):
    comment = re.sub(r"[\*\"“”\n\\…\+\-\/\=\(\)‘•:\[\]\|’\!;]", " ", str(comment))
    comment = re.sub(r"[ ]+", " ", comment)
    comment = re.sub(r"\!+", "!", comment)
    comment = re.sub(r"\,+", ",", comment)
    comment = re.sub(r"\?+", "?", comment)
    if len(comment) > MAX_CHARS:
        comment = comment[:MAX_CHARS]
    return [x.text for x in NLP.tokenizer(comment) if x.text != ' ']

In [3]:
import logging
import torch
from torchtext import data

LOGGER = logging.getLogger('toxic_dataset')

def get_dataset(fix_length=100, lower=False , vectors=None):
    if vectors is not None:
        # pretrain vectors only supports all lower case
        lower = True
    LOGGER.debug('Preparing CSV files...')
    prepare_csv()
    comment = data.Field(sequential=True, fix_length=fix_length, tokenize=tokenizer,
                        pad_first=true, tensor_type=torch.cuda.LongTensor, lower=lower
                    )
    LOGGER.debug('Reading train csv file...')
    train, val = data.TabularDataset.splits(path='cache/', format='csv', skip_header=True,
                        train='dataset_train.csv', validation='dataset_val.csv', fields=[
                            ('id', None),
                            ('comment_text', comment),
                            ('toxic', data.Field(use_vocab=False, sequential=False, tensor_type=torch.cuda.ByteTensor)),
                            ('severe_toxic', data.Field(use_vocab=False, sequential=False, tensor_type=torch.cuda.ByteTensor)),
                            ('obscene', data.Field(use_vocab=False, sequential=False, tensor_type=torch.cuda.ByteTensor)),
                            ('threat', data.Field(use_vocab=False, sequential=False, tensor_type=torch.cuda.ByteTensor)),
                            ('insult', data.Field(use_vocab=False, sequential=False, tensor_type=torch.cuda.ByteTensor)),
                            ('identit_hate', data.Field(use_vocab=False, sequential=False, tensor_type=torch.cuda.ByteTensor))
                        ])
    
    LOGGER.debug('Reading text csv file...')
    test = data.TabularDataset(path='cache/dataset_test.csv', format='csv',
                    skip_header=True, fields=[
                        ('id', None),
                        ('comment_text', comment)
                    ])
    LOGGER.debug('Building vocabulary...')
    comment.build_vocab(
        train, val, test,
        max_size=20000,
        min_freq=50,
        vectors=vectors
    )
    LOGGER.debug('Done preparing the datasets')
    return train, val, test
    

In [4]:
def get_iterator(dataset, batch_size, train=True, shuffle=True, repeat=False):
    dataset_iter = data.Iterator(
        dataset, batch_size=batch_size, device=0,
        train=train, shuffle=shuffle, repeat=repeat, sort=False
    )
    return dataset_iter

In [5]:
import re
import logging

import numpy as np
import pandas as pd
import spacy
import torch
from joblib import Memory
from torchtext import data
from sklearn.model_selection import KFold

NLP = spacy.load('en')
MAX_CHARS = 20000
LOGGER = logging.getLogger("toxic_dataset")
MEMORY = Memory(cachedir="cache/", verbose=1)


def tokenizer(comment):
    comment = re.sub(
        r"[\*\"“”\n\\…\+\-\/\=\(\)‘•:\[\]\|’\!;]", " ", str(comment))
    comment = re.sub(r"[ ]+", " ", comment)
    comment = re.sub(r"\!+", "!", comment)
    comment = re.sub(r"\,+", ",", comment)
    comment = re.sub(r"\?+", "?", comment)
    if (len(comment) > MAX_CHARS):
        comment = comment[:MAX_CHARS]
    return [x.text for x in NLP.tokenizer(comment) if x.text != " "]


def prepare_csv():
    df_train = pd.read_csv("data/train.csv")
    df_train["comment_text"] = df_train.comment_text.str.replace("\n", " ")
    df_train.to_csv("cache/dataset_train.csv", index=False)
    df_test = pd.read_csv("data/test.csv")
    df_test["comment_text"] = df_test.comment_text.str.replace("\n", " ")
    df_test.to_csv("cache/dataset_test.csv", index=False)


@MEMORY.cache
def read_files(fix_length=100, lower=False, vectors=None):
    if vectors is not None:
        # pretrain vectors only support all lower case
        lower = True
    LOGGER.debug("Preparing CSV files...")
    prepare_csv()
    comment = data.Field(
        sequential=True,
        fix_length=fix_length,
        tokenize=tokenizer,
        pad_first=True,
        tensor_type=torch.cuda.LongTensor,
        lower=lower
    )
    LOGGER.debug("Reading train csv file...")
    train = data.TabularDataset(
        path='cache/dataset_train.csv', format='csv', skip_header=True,
        fields=[
            ('id', None),
            ('comment_text', comment),
            ('toxic', data.Field(
                use_vocab=False, sequential=False, tensor_type=torch.cuda.ByteTensor)),
            ('severe_toxic', data.Field(
                use_vocab=False, sequential=False, tensor_type=torch.cuda.ByteTensor)),
            ('obscene', data.Field(
                use_vocab=False, sequential=False, tensor_type=torch.cuda.ByteTensor)),
            ('threat', data.Field(
                use_vocab=False, sequential=False, tensor_type=torch.cuda.ByteTensor)),
            ('insult', data.Field(
                use_vocab=False, sequential=False, tensor_type=torch.cuda.ByteTensor)),
            ('identity_hate', data.Field(
                use_vocab=False, sequential=False, tensor_type=torch.cuda.ByteTensor)),
        ])
    LOGGER.debug("Reading test csv file...")
    test = data.TabularDataset(
        path='cache/dataset_test.csv', format='csv', skip_header=True,
        fields=[
            ('id', None),
            ('comment_text', comment)
        ])
    LOGGER.debug("Building vocabulary...")
    comment.build_vocab(
        train, test,
        max_size=20000,
        min_freq=50,
        vectors=vectors
    )
    LOGGER.debug("Done preparing the datasets")
    return train.examples, test.examples, comment


def get_dataset(fix_length=100, lower=False, vectors=None, n_folds=5, seed=999):
    train_exs, test_exs, comment = read_files(
        fix_length=fix_length, lower=lower, vectors=vectors)

    kf = KFold(n_splits=n_folds, random_state=seed)

    fields = [
        ('id', None),
        ('comment_text', comment),
        ('toxic', data.Field(
            use_vocab=False, sequential=False, tensor_type=torch.cuda.ByteTensor)),
        ('severe_toxic', data.Field(
            use_vocab=False, sequential=False, tensor_type=torch.cuda.ByteTensor)),
        ('obscene', data.Field(
            use_vocab=False, sequential=False, tensor_type=torch.cuda.ByteTensor)),
        ('threat', data.Field(
            use_vocab=False, sequential=False, tensor_type=torch.cuda.ByteTensor)),
        ('insult', data.Field(
            use_vocab=False, sequential=False, tensor_type=torch.cuda.ByteTensor)),
        ('identity_hate', data.Field(
            use_vocab=False, sequential=False, tensor_type=torch.cuda.ByteTensor)),
    ]

    def iter_folds():
        train_exs_arr = np.array(train_exs)
        for train_idx, val_idx in kf.split(train_exs_arr):
            yield (
                data.Dataset(train_exs_arr[train_idx], fields),
                data.Dataset(train_exs_arr[val_idx], fields),
            )

    test = data.Dataset(test_exs, fields[:2])
    return iter_folds(), test


def get_iterator(dataset, batch_size, train=True, shuffle=True, repeat=False):
    dataset_iter = data.Iterator(
        dataset, batch_size=batch_size, device=0,
        train=train, shuffle=shuffle, repeat=repeat,
        sort=False
    )
    return dataset_iter

/home/phua/.virtualenvs/torchtext_load_nlp/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: The 'cachedir' parameter has been deprecated in version 0.12 and will be removed in version 0.14.
You provided "cachedir='cache/'", use "location='cache/'" instead.
  from ipykernel import kernelapp as app
